<a href="https://colab.research.google.com/github/Luixy0/PracticaWebService2/blob/master/final_P_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica 2

**Nombre:** Luixandra Azereth Saucedo Quirino  
**e-mail:** luixandra.saucedo8827@alumnos.udg.mx

#MODULES

In [5]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

#CLASSES

In [6]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

#FUNCTIONS

In [7]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

In [8]:
################################################################
#Brownian Motion Trajectory
################################################################

def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Return:
    BM_2d_df: Dataframe BM trajectory
  """
  velocity = Vec2d(speed,0)

  BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos':s_pos[1]}])
  BM_2d_df = pd.concat([BM_2d_df, temp_df],ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity =velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x,'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

# Actividad 1: Path Length - (BM1 vs BM2 vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Calcular métrica utilizando exclusivamente funciones de NumPy
* Guardar métricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [9]:
# Load existing trajectories
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [30]:
# Compute path length
## start - Add your code here
#Compute path-length with distance.euclidean
dis_BM_3 = np.array([np.sqrt(np.sum(np.square(BM_2d_df_3.iloc[i-1]-BM_2d_df_3.iloc[i])))
                    for i in range (1,BM_2d_df_3.shape[0])])
pl_BM_3= np.cumsum(dis_BM_3)

#Compute path-length with distance.euclidean
dis_BM_6 = np.array([np.sqrt(np.sum(np.square(BM_2d_df_6.iloc[i-1]-BM_2d_df_6.iloc[i])))
                    for i in range (1,BM_2d_df_6.shape[0])])
pl_BM_6= np.cumsum(dis_BM_6)


#Compute path-length with distance.euclidean
dis_CRW_6 = np.array([np.sqrt(np.sum(np.square(CRW_2d_df_9.iloc[i-1]-CRW_2d_df_9.iloc[i])))
                    for i in range (1,CRW_2d_df_9.shape[0])])
pl_CRW_6= np.cumsum(dis_CRW_6)
## end - Add your code here

In [31]:
#BM_2d_df_3.shape[0]
#i=100
#dis_BM_3
np.cumsum(dis_BM_3)

array([   3.,    6.,    9.,   12.,   15.,   18.,   21.,   24.,   27.,
         30.,   33.,   36.,   39.,   42.,   45.,   48.,   51.,   54.,
         57.,   60.,   63.,   66.,   69.,   72.,   75.,   78.,   81.,
         84.,   87.,   90.,   93.,   96.,   99.,  102.,  105.,  108.,
        111.,  114.,  117.,  120.,  123.,  126.,  129.,  132.,  135.,
        138.,  141.,  144.,  147.,  150.,  153.,  156.,  159.,  162.,
        165.,  168.,  171.,  174.,  177.,  180.,  183.,  186.,  189.,
        192.,  195.,  198.,  201.,  204.,  207.,  210.,  213.,  216.,
        219.,  222.,  225.,  228.,  231.,  234.,  237.,  240.,  243.,
        246.,  249.,  252.,  255.,  258.,  261.,  264.,  267.,  270.,
        273.,  276.,  279.,  282.,  285.,  288.,  291.,  294.,  297.,
        300.,  303.,  306.,  309.,  312.,  315.,  318.,  321.,  324.,
        327.,  330.,  333.,  336.,  339.,  342.,  345.,  348.,  351.,
        354.,  357.,  360.,  363.,  366.,  369.,  372.,  375.,  378.,
        381.,  384.,

In [32]:
# Plotting
# Init figure
fig_path_length = go.Figure()

#first TRACE BM_3
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_3)),
    y=pl_BM_3,
    marker = dict(size=2),
    line= dict(color='purple',width=2),
    mode='lines',
    name='path_length_BM_3',
    showlegend=True
))
    
## end - Add your code here


# Second trace BM2
## start - Add your code here
#first BM_6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_6)),
    y=pl_BM_6,
    marker = dict(size=2),
    line= dict(color='black',width=8),
    mode='lines',
    name='path_length_BM_6',
    showlegend=True
))
    
## end - Add your code here


# Third trace CRW
## start - Add your code here

#first CRW_6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_CRW_6)),
    y=pl_CRW_6,
    marker = dict(size=2),
    line= dict(color='pink',width=4),
    mode='lines',
    name='path_length_CRW_6',
    showlegend=True
))

fig_path_length.show()
## end - Add your code here

# Actividad 2: Mean Squared Displacement - (Brownian vs CRW)

---



* Cargar trayectorias en **Pandas** Data Frame
* Guardar metricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [13]:
# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [14]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [15]:
# Empty MSD_BM
MSD_BM = np.empty(shape=(0))
# Empty MSD_BM
# MSD for BM_2d_df_6
for tau in range(1,BM_2d_df_6.shape[0]):
    ## start - Add your code here
    displacement_vec = np.array([np.sqrt(np.sum(np.square(BM_2d_df_6.iloc[i-tau] - BM_2d_df_6.iloc[i])))
    for i in range (tau, BM_2d_df_6.shape[0],1)])
    displacement_vec= np.array(displacement_vec)
    np.reshape= (displacement_vec,(1, -1))
    dis_vec_square = displacement_vec*displacement_vec
    dvp_BM = np.mean(dis_vec_square)
    MSD_BM = np.append(MSD_BM,dvp_BM)
    ## end - Add your code here
  # Empty MSD_CRW
MSD_CRW = np.empty(shape=(0))
# MSD for CRW_2d_df_9
for tau in range(1,CRW_2d_df_9.shape[0]):
        ## start - Add your code here
        displacement_vec = np.array([np.sqrt(np.sum(np.square(CRW_2d_df_9.iloc[i-tau] - CRW_2d_df_9.iloc[i])))
        for i in range (tau, CRW_2d_df_9.shape[0],1)])
        displacement_vec= np.array(displacement_vec)
        np.reshape= (displacement_vec,(1, -1))
        dis_vec_square = displacement_vec*displacement_vec
        dvp_CRW = np.mean(dis_vec_square)
        MSD_CRW = np.append(MSD_CRW,dvp_CRW)
## start - Add your code here
## start - Add your code here
MSD_CRW_df= pd.DataFrame()
MSD_CRW_df['MSD_BM']=MSD_BM
temp_df= pd.DataFrame()
temp_df['MSD_CRW']=MSD_CRW
MSD_CRW_df_1=pd.merge(MSD_CRW_df,temp_df, left_index=True, right_index=True,how='inner')
MSD_CRW_df_1.to_csv("./MSD_CRW_F.csv")

In [16]:
MSD_CRW_df_1

,MSD_BM,MSD_CRW
0,36.000000,3.600000e+01
1,74.422372,1.368477e+02
2,112.979349,2.966932e+02
3,154.127854,5.100141e+02
4,193.378233,7.720513e+02
...,...,...
994,98997.081835,2.512414e+06
995,97496.121246,2.520607e+06
996,96630.627308,2.529127e+06
997,97027.421373,2.539040e+06


In [17]:
displacement_vec

array([1597.43863908])

In [33]:
# Init figure
fig_MSD=go.Figure()

# first trace MSD_BM
## start - Add your code here
#plotting
#first BM_6
fig_MSD.add_trace(go.Scatter(
    x = MSD_CRW_df_1.index,
    y = MSD_CRW_df_1.MSD_BM,
    marker = dict(size=2),
    line= dict(width=2),
    mode='lines',
    name='MSD BM 6',
    showlegend=True
))
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
#first CRW_6
fig_MSD.add_trace(go.Scatter(
    x = MSD_CRW_df_1.index,
    y = MSD_CRW_df_1.MSD_CRW,
    marker = dict(size=2),
    line= dict(width=2),
    mode='lines',
    name='MSD CRW 6 C=0.9',
    showlegend=True
))    
## end - Add your code here


fig_MSD.show()

# Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada)

* Generar CRWs con dos exponentes diferentes
* Guardar trayectorias en Pandas Data Frame
* Obtener Turning-angle distribution
* Guardar metricas en **Pandas** Data Frame
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [19]:
# Load existing trajectories
# CRW speed = 6, 
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories
# CRW speed = 6, 
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [20]:
aux_ta_CRW_2d_df_6 = np.empty(shape=(0))
aux_ta_CRW_2d_df_9 = np.empty(shape=(0))

for index, row in CRW_2d_df_6[1:-1].iterrows():        
                a= CRW_2d_df_6.iloc[index-1]
                b= CRW_2d_df_6.iloc[index]
                c= CRW_2d_df_6.iloc[index+1]
                crw6 = turning_angle(a,b,c)
                aux_ta_CRW_2d_df_6 = np.append(aux_ta_CRW_2d_df_6,crw6)
                
for index, row in CRW_2d_df_9[1:-1].iterrows():        
                a=CRW_2d_df_9.iloc[index -1]
                b= CRW_2d_df_9.iloc[index]
                c=CRW_2d_df_9.iloc[index +1]
                crw9= turning_angle(a,b,c)
                aux_ta_CRW_2d_df_9 = np.append(aux_ta_CRW_2d_df_9,crw9)
CRW_6_9_df= pd.DataFrame()
CRW_6_9_df['TA_CRW_6']=aux_ta_CRW_2d_df_6
temp_df= pd.DataFrame()
temp_df['TA_CRW_9']=aux_ta_CRW_2d_df_9
CRW1_6_9_df=pd.merge(CRW_6_9_df,temp_df, left_index=True, right_index=True,how='inner')


In [21]:
CRW1_6_9_df

,TA_CRW_6,TA_CRW_9
0,1.526081,0.131243
1,-1.995115,-0.116685
2,1.248337,0.310765
3,0.478471,-0.086050
4,-0.547363,1.131171
...,...,...
993,-0.615127,-0.199832
994,-0.762372,-0.877469
995,0.024496,0.140115
996,-0.637645,0.044725


In [37]:
# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_CRW_2d_df_6= go.Figure()


fig_CRW_2d_df_6.add_trace(go.Histogram(x=CRW1_6_9_df.TA_CRW_6,
                                    xbins=dict(size=np.pi/50),
                                    name='CRW_0.6',
                                    opacity=0.75,
                                    marker_color='#330C73',
                                    showlegend=True
                                    ))
fig_CRW_2d_df_6.update_layout(
    title_text='CRW_06 Results', # title of plot
    xaxis_title_text='TA_CRW_6', # xaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig_CRW_2d_df_6.show()
## end - Add your code here

In [38]:
# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig_CRW_2d_df_9 = go.Figure()

fig_CRW_2d_df_9.add_trace(go.Histogram(x=CRW1_6_9_df.TA_CRW_9,
                                    xbins=dict(size=np.pi/50),
                                    name='CRW_0.9',
                                    opacity=0.75,
                                    marker_color='#EB89B5',
                                    showlegend=True
                                    ))
fig_CRW_2d_df_9.update_layout(
    title_text='CRW_09 Results', # title of plot
    xaxis_title_text='TA_CRW_9', # xaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig_CRW_2d_df_9.show()
## end - your code here


In [39]:
# Check documentation
# https://plotly.com/python/histograms/
# PLot histogram
# Histogram turning angle CRW_2d_df_9

fig_CRW_06_09 = go.Figure()
fig_CRW_06_09.add_trace(go.Histogram(x=CRW1_6_9_df.TA_CRW_6,
                                    xbins=dict(size=np.pi/300),
                                    histnorm='percent',
                                    name='CRW_0.6',
                                    opacity=0.85,
                                    marker_color='#EB89B5',
                                    showlegend=True
                                    ))
fig_CRW_06_09.update_layout(
    title_text='CRW_06 Results', # title of plot
    xaxis_title_text='TA_CRW_6', # xaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig_CRW_06_09.add_trace(go.Histogram(x=CRW1_6_9_df.TA_CRW_9,
                                    xbins=dict(size=np.pi/300),
                                    histnorm='percent',
                                    name='CRW_0.9',
                                    opacity=0.60,
                                    marker_color='skyblue',
                                    showlegend=True
                                    ))
fig_CRW_06_09.update_layout(
    title_text='CRW_09 Results', # title of plot
    xaxis_title_text='TA_CRW_9', # xaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
#Resolution
CRW_exponent = 0.1
resolution = 200
aux_domain = np.linspace(0, 2*np.pi, resolution)
for CRW_exponent in [0.6, 0.9]:
  wrapcauchy_pdf = np.array([wrapcauchy.pdf(i,CRW_exponent) for i in aux_domain])
  #Another aux vector
  aux_plot = np.linspace(-np.pi, np.pi, resolution)
  #Only for plotting
  plot_wrapcauchy_pdf =np.concatenate((wrapcauchy_pdf[int(resolution/2):resolution],wrapcauchy_pdf[0:int(resolution/2)]), axis=0)

  fig_CRW_06_09.add_trace(go.Scatter(x = aux_plot, 
                                        y =plot_wrapcauchy_pdf,
                                        marker = dict(size=2),
                                        line = dict(width=2),
                                        mode = 'lines',
                                        name = 'cauchy_{}'.format(CRW_exponent),
                                        showlegend = True))
fig_CRW_06_09.show()

# Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada)

* Generar Levy Flight
* Guardar trayectorias en **Pandas** Data Frame
* Guardar metricas en **numpy** array
* Obtener Step-length distribution
* Comparar en gráfica distribución origen vs distribución observada
* Visualizar con **plotly**

In [43]:
# Load existing trajectories
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [49]:
# aux to store turning angles
aux_ta_Levy_2d_df_1 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_1 = np.empty(shape=(0))

beta=1.0
alpha=1.0
loc=3.0
resolution= 200
prev_index = 0

aux_domain = np.linspace(0,50,resolution)
## start - Add your code here

for index, row in Levy_2d_df_1[1:-1].iterrows():
            a= Levy_2d_df_1.iloc[index-1]
            b= Levy_2d_df_1.iloc[index]
            c= Levy_2d_df_1.iloc[index+1]
            LW_1 = turning_angle(a,b,c)
                      
            if (LW_1 != 0.0):
              aux_ta_Levy_2d_df_1 = np.append(aux_ta_Levy_2d_df_1,LW_1)
              levyStep =  distance.euclidean( Levy_2d_df_1.iloc[prev_index] , Levy_2d_df_1.iloc[index] )
              levyStep1=levyStep/6
              aux_sl_Levy_2d_df_1 = np.concatenate ( (aux_sl_Levy_2d_df_1, [levyStep1]) , axis=0)
              prev_index = index
       
                         
Levy_pdf = np.array([levy_stable.pdf(i,alpha,beta,loc)for i in aux_domain])
for LDF in range(Levy_pdf.shape[0]):                  
              LW_TA_SL_df= pd.DataFrame()
              LW_TA_SL_df['LW_TA']=aux_ta_Levy_2d_df_1
              temp_df= pd.DataFrame()
              temp_df['LW_SL']=aux_sl_Levy_2d_df_1 
              LW1_TA_SL_df=pd.merge(LW_TA_SL_df,temp_df, left_index=True, right_index=True,how='inner')

In [50]:
# Histogram turning angle LW_2d_df_TA
## start - Add your code here
fig_LW2_2d_df_TA= go.Figure()


fig_LW2_2d_df_TA.add_trace(go.Histogram(x=LW1_TA_SL_df.LW_SL,
                                    xbins=dict(start= 0, end=50, size=1),
                                    histnorm='probability density',
                                    name='LW2_LS',
                                    opacity=0.75,
                                    marker_color='#EB89B5',
                                    showlegend=True
                                    ))
fig_LW2_2d_df_TA.update_layout(
    title_text='LW2_LS Results', # title of plot
    xaxis_title_text='LW2_LS', # xaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig_LW2_2d_df_TA.show()
## end - Add your code here



In [51]:
# aux to store turning angles
aux_ta_Levy_2d_df_7 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_7 = np.empty(shape=(0))

beta=1.0
alpha=0.7
loc=3.0
resolution= 200
prev_index = 0

aux_domain = np.linspace(0,50,resolution)
for index, row in Levy_2d_df_7[1:-1].iterrows():
            a= Levy_2d_df_7.iloc[index-1]
            b= Levy_2d_df_7.iloc[index]
            c= Levy_2d_df_7.iloc[index+1]
            LW_2 = turning_angle(a,b,c)
         
            if (LW_2 != 0.0):
              aux_ta_Levy_2d_df_7 = np.append(aux_ta_Levy_2d_df_7,LW_2)
              levyStep =  distance.euclidean( Levy_2d_df_7.iloc[prev_index] , Levy_2d_df_7.iloc[index])
              levyStep1=levyStep/6
              aux_sl_Levy_2d_df_7 = np.concatenate ( (aux_sl_Levy_2d_df_7, [levyStep1]) , axis=0)
              prev_index = index                    
                         
for LDF in range(Levy_pdf.shape[0]):                  
              LW2_TA_SL_df= pd.DataFrame()
              LW2_TA_SL_df['LW_TA']=aux_ta_Levy_2d_df_7
              temp_df= pd.DataFrame()
              temp_df['LW_SL']=aux_sl_Levy_2d_df_7 
              LW22_TA_SL_df=pd.merge(LW2_TA_SL_df,temp_df, left_index=True, right_index=True,how='inner')
## end - Add your code here


In [53]:
# Histogram turning angle LW_2d_df_TA
## start - Add your code here
fig_LW2_2d_df_TA= go.Figure()


fig_LW2_2d_df_TA.add_trace(go.Histogram(x=LW22_TA_SL_df.LW_SL,
                                    xbins=dict(start= 0, end=50, size=1),
                                    histnorm='probability density',
                                    name='LW2_LS',
                                    opacity=0.75,
                                    marker_color='#EB89B5',
                                    showlegend=True
                                    ))
fig_LW2_2d_df_TA.update_layout(
    title_text='LW2_LS Results', # title of plot
    xaxis_title_text='LW2_LS', # xaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig_LW2_2d_df_TA.show()
## end - Add your code here


In [52]:
#Plot levy_stable
fig_LW_TA_LS= go.Figure()


  
fig_LW_TA_LS.add_trace(go.Histogram(x=LW1_TA_SL_df.LW_SL,
                                    xbins=dict(start= 0, end=50, size=1),
                                    histnorm='probability density',
                                    name='LW_LS',
                                    opacity=0.75,
                                    marker_color='#330C73',
                                    showlegend=True
                                    ))


fig_LW_TA_LS.update_layout(
    title_text='LW1_LS Results', # title of plot
    xaxis_title_text='LW1_LS', # xaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)


fig_LW_TA_LS.add_trace(go.Histogram(x=LW22_TA_SL_df.LW_SL,
                                    xbins=dict(start= 0, end=50, size=1),
                                    histnorm='probability density',
                                    name='LW2_LS',
                                    opacity=0.75,
                                    marker_color='#EB89B5',
                                    showlegend=True
                                    ))
fig_LW_TA_LS.update_layout(
    title_text='LW2_LS Results', # title of plot
    xaxis_title_text='LW2_LS', # xaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

#Add origin distributions
loc= 5.0
beta= 1.0
resolution= 200
aux_domain = np.linspace(0,50,resolution)

for Levy_exponent in [1.0,0.7]:
  Levy_pdf=np.array([levy_stable.pdf(i,Levy_exponent,beta,loc)for i in aux_domain])

  fig_LW_TA_LS.add_trace(go.Scatter(x=aux_domain,
                                    y=Levy_pdf,
                                    line=dict(width=2),
                                    mode='lines',
                                    name='Levy_{}'.format(Levy_exponent),
                                    showlegend=True
                                    ))



fig_LW_TA_LS.show()